데이터셋 출처 : https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset/data

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import contractions

In [2]:
### 45466개 행 존재
metaDF = pd.read_csv('../data/movielens_dataset/movies_metadata.csv')
metaDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

C:\Users\kdp\AppData\Local\Temp\ipykernel_21508\62695679.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metaDF = pd.read_csv('../data/movielens_dataset/movies_metadata.csv')


In [3]:
metaDF.iloc[:, [0, 1, 3, 5, 6, 9, 11, 14]].head(2)

,adult,belongs_to_collection,genres,id,imdb_id,overview,poster_path,release_date
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,"Led by Woody, Andy's toys live happily in his ...",/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30
1,False,NaN,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,tt0113497,When siblings Judy and Peter discover an encha...,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15


In [4]:
### 메모리 문제로 인해, 25000개 영화만을 다룰 것
metaDF = metaDF.head(25000)

#### overview 컬럼을 가지고 유사도를 측정할 것이므로, 결측치 조사

In [5]:
### overview 컬럼의 결측치 유무 조사 (954 개 존재)
metaDF['overview'].isnull().sum()

209

In [6]:
### 결측값을 빈 문자열값으로 대체
metaDF['overview'] = metaDF['overview'].fillna('')

In [7]:
### 결측치 제거 확인
metaDF['overview'].isnull().sum()

0

#### 영어 축약어를 원래대로 되돌리는 전처리 수행

In [8]:
### 예시 문장
metaDF.loc[2, 'overview']

"A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max."

In [9]:
### contractions 라이브러리를 사용해 축약어 확장 실행
metaDF['overview'] = metaDF['overview'].apply(lambda x: contractions.fix(x))

In [10]:
metaDF.loc[2, 'overview']

'A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she will scare the fish away. But she is less interested in seafood than she is in cooking up a hot time with Max.'

In [11]:
### TF-IDF 행렬 생성 (=> 53090 단어 존재)
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(metaDF['overview'])  # scipy.sparse._csr.csr_matrix
print(f'TF-IDF 행렬의 크기(shape) : {tfidf_matrix.shape}')

TF-IDF 행렬의 크기(shape) : (25000, 53090)


In [12]:
### 메모리 부족 주의 !!! (MemoryError)
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)  # ndarray
print(f'코사인 유사도 연산 결과 : {cosine_sim.shape}')

코사인 유사도 연산 결과 : (25000, 25000)


In [13]:
cosine_sim.nbytes

5000000000

In [14]:
### 영화제목 -> 해당 인덱스 (딕셔너리)
title_to_index = dict(zip(metaDF['title'], metaDF.index))

In [20]:
def get_recommendations(title, cosine_sim):
    ### 선택한 영화의 제목으로부터 해당 영화의 인덱스를 받아온다.
    idx = title_to_index[title]
    
    ### 해당 영화와 모든 영화와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    ### 유사도에 따라 영화들을 정렬한다.
    sim_scores.sort(key=lambda x: x[1], reverse=True)
    
    ### 가장 유사한 10개의 영화를 추출한다.
    sim_scores = sim_scores[1:11]
    
    ### 가장 유사한 10개 영화의 인덱스를 얻는다.
    movie_indices = [idx[0] for idx in sim_scores]
    
    ### 가장 유사한 10개 영화의 제목을 리턴한다.
    return metaDF['title'].iloc[movie_indices]
    # return movie_indices

In [21]:
get_recommendations('The Dark Knight Rises', cosine_sim=cosine_sim)

12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
585                                                 Batman
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
18035                                     Batman: Year One
19792              Batman: The Dark Knight Returns, Part 1
3095                          Batman: Mask of the Phantasm
Name: title, dtype: object

In [22]:
### float64 -> float16 으로 변환
cosin_sim = cosine_sim.astype(np.float16)

In [23]:
get_recommendations('The Dark Knight Rises', cosine_sim=cosine_sim)

12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
585                                                 Batman
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
18035                                     Batman: Year One
19792              Batman: The Dark Knight Returns, Part 1
3095                          Batman: Mask of the Phantasm
Name: title, dtype: object

In [24]:
### 코사인 유사도 행렬(ndarray)을 csv 파일로 저장 (float16으로 저장해도 10GB가 넘는다.)
np.savetxt('../data/processed/cosine_similarity_25000.csv', cosine_sim, delimiter=',')

줄거리 기준 유사도 행렬 + 배우/감독 기준 유사도 행렬 결과를 저장할까?